In [ ]:
'''
This code is based on
https://github.com/hunkim/DeepRL-Agents
'''
import numpy as np
import tensorflow as tf
from collections import deque

import gym
env = gym.make('CartPole-v0')

# Constants defining our neural network
learning_rate = 1e-1
input_size = env.observation_space.shape[0]
output_size = env.action_space.n

X = tf.placeholder(tf.float32, [None, input_size], name="input_x")

# First layer of weights
W1 = tf.get_variable("W1", shape=[input_size, output_size],
                     initializer=tf.contrib.layers.xavier_initializer())
Qpred = tf.matmul(X, W1)

# We need to define the parts of the network needed for learning a policy
Y = tf.placeholder(shape=[None, output_size], dtype=tf.float32)

# Loss function
loss = tf.reduce_sum(tf.square(Y - Qpred))
# Learning
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Values for q learning
max_episodes = 5000
dis = 0.9
step_history = []


# Setting up our environment
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for episode in range(max_episodes):
    e = 1. / ((episode / 10) + 1)
    step_count = 0
    state = env.reset()
    done = False

    # The Q-Network training
    while not done:
        step_count += 1
        x = np.reshape(state, [1, input_size])
        # Choose an action by greedily (with e chance of random action) from
        # the Q-network
        Q = sess.run(Qpred, feed_dict={X: x})
        if np.random.rand(1) < e:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q)

        # Get new state and reward from environment
        next_state, reward, done, _ = env.step(action)
        if done:
            Q[0, action] = -100
        else:
            x_next = np.reshape(next_state, [1, input_size])
            # Obtain the Q' values by feeding the new state through our network
            Q_next = sess.run(Qpred, feed_dict={X: x_next})
            Q[0, action] = reward + dis * np.max(Q_next)

        # Train our network using target and predicted Q values on each episode
        sess.run(train, feed_dict={X: x, Y: Q})
        state = next_state

    step_history.append(step_count)
    print("Episode: {}  steps: {}".format(episode, step_count))
    # If last 10's avg steps are 500, it's good enough
    if len(step_history) > 10 and np.mean(step_history[-10:]) > 500:
        break

# See our trained network in action
observation = env.reset()
reward_sum = 0
while True:
    env.render()

    x = np.reshape(observation, [1, input_size])
    Q = sess.run(Qpred, feed_dict={X: x})
    action = np.argmax(Q)

    observation, reward, done, _ = env.step(action)
    reward_sum += reward
    if done:
        print("Total score: {}".format(reward_sum))
        break


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Episode: 0  steps: 16
Episode: 1  steps: 9
Episode: 2  steps: 17
Episode: 3  steps: 21
Episode: 4  steps: 17
Episode: 5  steps: 45
Episode: 6  steps: 38
Episode: 7  steps: 42
Episode: 8  steps: 51
Episode: 9  steps: 64
Episode: 10  steps: 52
Episode: 11  steps: 16
Episode: 12  steps: 80
Episode: 13  steps: 40
Episode: 14  steps: 35
Episode: 15  steps: 35
Episode: 16  steps: 61
Episode: 17  steps: 31
Episode: 18  steps: 8
Episode: 19  steps: 73
Episode: 20  steps: 33
Episode: 21  steps: 17
Episode: 22  steps: 9
Episode: 23  steps: 9
Episode: 24  steps: 14
Episode: 25  steps: 14
Episode: 26  steps: 33
Episode: 27  steps: 44
Episode: 28  steps: 55
Episode: 29  steps: 44
Episode: 30  steps: 36
Episode: 31  steps: 30
Episode: 32  steps: 56
Episode: 33  steps: 38
Episode: 34  steps: 41
Episode: 35  steps: 24
Episode: 36  steps: 29
Episode: 37  steps: 20
Episode: 38  steps: 23
Episode: 39  steps

Episode: 344  steps: 32
Episode: 345  steps: 38
Episode: 346  steps: 37
Episode: 347  steps: 24
Episode: 348  steps: 38
Episode: 349  steps: 58
Episode: 350  steps: 63
Episode: 351  steps: 26
Episode: 352  steps: 21
Episode: 353  steps: 17
Episode: 354  steps: 16
Episode: 355  steps: 51
Episode: 356  steps: 24
Episode: 357  steps: 64
Episode: 358  steps: 32
Episode: 359  steps: 26
Episode: 360  steps: 56
Episode: 361  steps: 22
Episode: 362  steps: 67
Episode: 363  steps: 27
Episode: 364  steps: 34
Episode: 365  steps: 39
Episode: 366  steps: 33
Episode: 367  steps: 37
Episode: 368  steps: 49
Episode: 369  steps: 32
Episode: 370  steps: 38
Episode: 371  steps: 48
Episode: 372  steps: 19
Episode: 373  steps: 28
Episode: 374  steps: 38
Episode: 375  steps: 37
Episode: 376  steps: 25
Episode: 377  steps: 63
Episode: 378  steps: 75
Episode: 379  steps: 31
Episode: 380  steps: 24
Episode: 381  steps: 21
Episode: 382  steps: 23
Episode: 383  steps: 10
Episode: 384  steps: 32
Episode: 385  st

Episode: 688  steps: 43
Episode: 689  steps: 23
Episode: 690  steps: 34
Episode: 691  steps: 44
Episode: 692  steps: 22
Episode: 693  steps: 27
Episode: 694  steps: 32
Episode: 695  steps: 19
Episode: 696  steps: 32
Episode: 697  steps: 33
Episode: 698  steps: 20
Episode: 699  steps: 11
Episode: 700  steps: 9
Episode: 701  steps: 12
Episode: 702  steps: 9
Episode: 703  steps: 9
Episode: 704  steps: 10
Episode: 705  steps: 12
Episode: 706  steps: 30
Episode: 707  steps: 13
Episode: 708  steps: 26
Episode: 709  steps: 18
Episode: 710  steps: 20
Episode: 711  steps: 34
Episode: 712  steps: 83
Episode: 713  steps: 36
Episode: 714  steps: 75
Episode: 715  steps: 21
Episode: 716  steps: 33
Episode: 717  steps: 42
Episode: 718  steps: 32
Episode: 719  steps: 38
Episode: 720  steps: 20
Episode: 721  steps: 39
Episode: 722  steps: 49
Episode: 723  steps: 22
Episode: 724  steps: 39
Episode: 725  steps: 24
Episode: 726  steps: 32
Episode: 727  steps: 32
Episode: 728  steps: 25
Episode: 729  steps

Episode: 1030  steps: 55
Episode: 1031  steps: 32
Episode: 1032  steps: 17
Episode: 1033  steps: 16
Episode: 1034  steps: 12
Episode: 1035  steps: 24
Episode: 1036  steps: 27
Episode: 1037  steps: 70
Episode: 1038  steps: 33
Episode: 1039  steps: 24
Episode: 1040  steps: 25
Episode: 1041  steps: 8
Episode: 1042  steps: 10
Episode: 1043  steps: 10
Episode: 1044  steps: 35
Episode: 1045  steps: 9
Episode: 1046  steps: 28
Episode: 1047  steps: 23
Episode: 1048  steps: 31
Episode: 1049  steps: 14
Episode: 1050  steps: 19
Episode: 1051  steps: 20
Episode: 1052  steps: 21
Episode: 1053  steps: 22
Episode: 1054  steps: 33
Episode: 1055  steps: 18
Episode: 1056  steps: 9
Episode: 1057  steps: 9
Episode: 1058  steps: 10
Episode: 1059  steps: 8
Episode: 1060  steps: 27
Episode: 1061  steps: 15
Episode: 1062  steps: 24
Episode: 1063  steps: 27
Episode: 1064  steps: 19
Episode: 1065  steps: 36
Episode: 1066  steps: 21
Episode: 1067  steps: 10
Episode: 1068  steps: 9
Episode: 1069  steps: 9
Episode

Episode: 1359  steps: 16
Episode: 1360  steps: 24
Episode: 1361  steps: 40
Episode: 1362  steps: 16
Episode: 1363  steps: 18
Episode: 1364  steps: 18
Episode: 1365  steps: 35
Episode: 1366  steps: 34
Episode: 1367  steps: 21
Episode: 1368  steps: 20
Episode: 1369  steps: 25
Episode: 1370  steps: 23
Episode: 1371  steps: 20
Episode: 1372  steps: 42
Episode: 1373  steps: 22
Episode: 1374  steps: 34
Episode: 1375  steps: 43
Episode: 1376  steps: 18
Episode: 1377  steps: 32
Episode: 1378  steps: 18
Episode: 1379  steps: 23
Episode: 1380  steps: 34
Episode: 1381  steps: 23
Episode: 1382  steps: 42
Episode: 1383  steps: 35
Episode: 1384  steps: 21
Episode: 1385  steps: 21
Episode: 1386  steps: 29
Episode: 1387  steps: 12
Episode: 1388  steps: 22
Episode: 1389  steps: 37
Episode: 1390  steps: 66
Episode: 1391  steps: 23
Episode: 1392  steps: 21
Episode: 1393  steps: 20
Episode: 1394  steps: 9
Episode: 1395  steps: 8
Episode: 1396  steps: 10
Episode: 1397  steps: 11
Episode: 1398  steps: 11
Ep

Episode: 1691  steps: 37
Episode: 1692  steps: 24
Episode: 1693  steps: 13
Episode: 1694  steps: 9
Episode: 1695  steps: 9
Episode: 1696  steps: 12
Episode: 1697  steps: 20
Episode: 1698  steps: 27
Episode: 1699  steps: 39
Episode: 1700  steps: 14
Episode: 1701  steps: 30
Episode: 1702  steps: 32
Episode: 1703  steps: 13
Episode: 1704  steps: 20
Episode: 1705  steps: 20
Episode: 1706  steps: 37
Episode: 1707  steps: 26
Episode: 1708  steps: 55
Episode: 1709  steps: 30
Episode: 1710  steps: 30
Episode: 1711  steps: 43
Episode: 1712  steps: 16
Episode: 1713  steps: 42
Episode: 1714  steps: 24
Episode: 1715  steps: 35
Episode: 1716  steps: 35
Episode: 1717  steps: 25
Episode: 1718  steps: 52
Episode: 1719  steps: 22
Episode: 1720  steps: 22
Episode: 1721  steps: 27
Episode: 1722  steps: 17
Episode: 1723  steps: 21
Episode: 1724  steps: 51
Episode: 1725  steps: 41
Episode: 1726  steps: 18
Episode: 1727  steps: 31
Episode: 1728  steps: 18
Episode: 1729  steps: 24
Episode: 1730  steps: 38
Ep

Episode: 2023  steps: 20
Episode: 2024  steps: 23
Episode: 2025  steps: 17
Episode: 2026  steps: 14
Episode: 2027  steps: 14
Episode: 2028  steps: 20
Episode: 2029  steps: 17
Episode: 2030  steps: 9
Episode: 2031  steps: 49
Episode: 2032  steps: 9
Episode: 2033  steps: 10
Episode: 2034  steps: 9
Episode: 2035  steps: 8
Episode: 2036  steps: 13
Episode: 2037  steps: 20
Episode: 2038  steps: 19
Episode: 2039  steps: 12
Episode: 2040  steps: 33
Episode: 2041  steps: 9
Episode: 2042  steps: 9
Episode: 2043  steps: 11
Episode: 2044  steps: 10
Episode: 2045  steps: 9
Episode: 2046  steps: 9
Episode: 2047  steps: 9
Episode: 2048  steps: 16
Episode: 2049  steps: 10
Episode: 2050  steps: 21
Episode: 2051  steps: 9
Episode: 2052  steps: 12
Episode: 2053  steps: 21
Episode: 2054  steps: 8
Episode: 2055  steps: 25
Episode: 2056  steps: 8
Episode: 2057  steps: 14
Episode: 2058  steps: 48
Episode: 2059  steps: 14
Episode: 2060  steps: 33
Episode: 2061  steps: 34
Episode: 2062  steps: 14
Episode: 206

Episode: 2355  steps: 35
Episode: 2356  steps: 29
Episode: 2357  steps: 14
Episode: 2358  steps: 54
Episode: 2359  steps: 19
Episode: 2360  steps: 15
Episode: 2361  steps: 18
Episode: 2362  steps: 10
Episode: 2363  steps: 12
Episode: 2364  steps: 9
Episode: 2365  steps: 9
Episode: 2366  steps: 9
Episode: 2367  steps: 26
Episode: 2368  steps: 18
Episode: 2369  steps: 18
Episode: 2370  steps: 13
Episode: 2371  steps: 11
Episode: 2372  steps: 17
Episode: 2373  steps: 21
Episode: 2374  steps: 10
Episode: 2375  steps: 10
Episode: 2376  steps: 13
Episode: 2377  steps: 9
Episode: 2378  steps: 9
Episode: 2379  steps: 9
Episode: 2380  steps: 21
Episode: 2381  steps: 8
Episode: 2382  steps: 10
Episode: 2383  steps: 9
Episode: 2384  steps: 9
Episode: 2385  steps: 10
Episode: 2386  steps: 10
Episode: 2387  steps: 9
Episode: 2388  steps: 9
Episode: 2389  steps: 23
Episode: 2390  steps: 15
Episode: 2391  steps: 16
Episode: 2392  steps: 12
Episode: 2393  steps: 30
Episode: 2394  steps: 8
Episode: 239

In [ ]:
# 왜학습이 잘 안될까?

# 1. 얋은 층
# 2. 샘플사이의 Correlations
# 3. Non-stationary targets 타겟이 정해져 있지 않다.